# Setup and simulation of Cairns example

** So, it seems anuga replaces stdout and the notebook does not like it, so as seen in [this post](http://stackoverflow.com/questions/25494182/print-not-showing-in-ipython-notebook-python), I store the original and restore it afterwards **

In [1]:
import sys
stdout = sys.stdout

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline 
plt.style.use('ggplot')
import anuga

In [3]:
sys.stdout = stdout

## Configuring the simulation

Files required:
* **extent.csv**, points that define the outer bounding polygon
* **cairns.csv**, points that define one of the inner regions of the grid
* **islands.csv**, points that define one of the inner regions of the grid
* **islands1.csv**, points that define one of the inner regions of the grid
* **islands2.csv**, points that define one of the inner regions of the grid
* **islands3.csv**, points that define one of the inner regions of the grid
* **shallow.csv**, points that define one of the inner regions of the grid
* **cairns.prj**, file that describes the UTM zone of the simulation

In [4]:
cache = False
verbose = True

In [5]:
name_stem = 'cairns'

* Polygon

In [6]:
bounding_polygon = anuga.read_polygon('extent.csv')

A = anuga.polygon_area(bounding_polygon) / 1000000.0
print 'Area of bounding polygon = %.2f km^2' % A

Area of bounding polygon = 107720.35 km^2


* Inside regions

In [7]:
poly_cairns = anuga.read_polygon('cairns.csv')
poly_island0 = anuga.read_polygon('islands.csv')
poly_island1 = anuga.read_polygon('islands1.csv')
poly_island2 = anuga.read_polygon('islands2.csv')
poly_island3 = anuga.read_polygon('islands3.csv')
poly_shallow = anuga.read_polygon('shallow.csv')

* Define resolutions (max area per triangle) for each polygon
* Make these numbers larger to reduce the number of triangles in the model, and hence speed up the simulation

In [8]:
# bigger base_scale == less triangles
just_fitting = False
#base_scale = 25000 # 635763 # 112sec fit
#base_scale = 50000 # 321403 # 69sec fit
base_scale = 100000 # 162170 triangles # 45sec fit
#base_scale = 400000 # 42093
default_res = 100 * base_scale   # Background resolution
islands_res = base_scale
cairns_res = base_scale
shallow_res = 5 * base_scale

In [9]:
# Define list of interior regions with associated rezsolutions
interior_regions = [[poly_cairns,  cairns_res],
                    [poly_island0, islands_res],
                    [poly_island1, islands_res],
                    [poly_island2, islands_res],
                    [poly_island3, islands_res],
                    [poly_shallow, shallow_res]]

In [10]:
#------------------------------------------------------------------------------
# Data for exporting ascii grid
#------------------------------------------------------------------------------
eastingmin = 363000
#eastingmax = 418000
eastingmax = 418000
northingmin = 8026600
northingmax = 8145700

# Running the simulation

In [11]:
import os
import time
import sys

## Preparation of topographic data

### Convert ASC to DEM to PTS using source data and store result in source data

* Unzip asc from zip file

In [15]:
import zipfile as zf
if verbose: print 'Reading ASC from cairns.zip'
zf.ZipFile('cairns.zip').extract('cairns.asc')

Reading ASC from cairns.zip


'/media/jose/Docs/github/anuga_examples/cairns/cairns.asc'

* Convert unzipped asc to dem, requires cairns.prj file

In [16]:
anuga.asc2dem('cairns.asc', use_cache=cache, 
              verbose=verbose)

Reading METADATA from cairns.prj
Reading DEM from cairns.asc
Got 1194 lines
Store to NetCDF file cairns.dem


* Create pts file for onshore DEM

In [17]:
anuga.dem2pts(name_stem+'.dem', use_cache=cache, 
              verbose=verbose)

Reading DEM from cairns.dem
Store to NetCDF file cairns.pts
There are 1881792 values in the elevation
There are 1881792 values in the clipped elevation
There are 18234 NODATA_values in the clipped elevation


## Create the triangular mesh and domain

In [18]:
domain = anuga.create_domain_from_regions(bounding_polygon,
                                    boundary_tags={'top': [0],
                                                   'ocean_east': [1],
                                                   'bottom': [2],
                                                   'onshore': [3]},
                                    maximum_triangle_area=default_res,
                                    mesh_filename='cairns.msh',
                                    interior_regions=interior_regions,
                                    use_cache=cache,
                                    verbose=verbose)

Generating mesh to file 'cairns.msh'
Domain: Initialising
Pmesh_to_Domain: Initialising
Pmesh_to_Domain: Done
General_mesh: Building basic mesh structure
General_mesh: Computing areas, normals, edgelengths, centroids and radii
General Mesh: Building inverted triangle structure
Mesh: Initialising
Mesh: Building neigbour structure
Mesh: Building surrogate neigbour structure
Mesh: Building boundary dictionary
Mesh: Building tagged elements dictionary
Mesh: Done
Domain: Expose mesh attributes
Domain: Expose quantity names and types
Domain: Build Quantities
Domain: Set up communication buffers 
Domain: Set up triangle/node full flags 
Domain: Set defaults
Domain: Set work arrays
Domain: Initialising quantity values
Domain: Done
##########################################################################
#
# Using discontinuous elevation solver DE0
#
# First order timestepping
#
# Make sure you use centroid values when reporting on important output quantities
#
################################

In [19]:
# Print some stats about mesh and domain
print 'Number of triangles = ', len(domain)
print 'The extent is ', domain.get_extent()
print domain.statistics()
                         

Number of triangles =  162170
The extent is  (0.0, 410914.80000000005, 0.0, 311171.37000000011)
------------------------------------------------
Mesh statistics:
  Number of triangles = 162170
  Extent [m]:
    x in [0.00000e+00, 4.10915e+05]
    y in [0.00000e+00, 3.11171e+05]
  Areas [m^2]:
    A in [2.20532e+04, 9.99835e+06]
    number of distinct areas: 162170
    Histogram:
      [2.20532e+04, 1.01968e+06[: 148143
      [1.01968e+06, 2.01731e+06[: 1026
      [2.01731e+06, 3.01494e+06[: 396
      [3.01494e+06, 4.01257e+06[: 404
      [4.01257e+06, 5.01020e+06[: 2173
      [5.01020e+06, 6.00783e+06[: 3260
      [6.00783e+06, 7.00546e+06[: 2674
      [7.00546e+06, 8.00309e+06[: 2116
      [8.00309e+06, 9.00072e+06[: 1256
      [9.00072e+06, 9.99835e+06]: 722
    Percentiles (10 percent):
      16217 triangles in [2.20532e+04, 5.03522e+04]
      16217 triangles in [5.03522e+04, 5.75329e+04]
      16217 triangles in [5.75329e+04, 6.57625e+04]
      16217 triangles in [6.57625e+04, 7.51

Setup parameters of computational domain

In [22]:
domain.set_name('cairns_fixed_wave') # Name of sww file
domain.set_datadir('.')                       # Store sww output here
domain.set_minimum_storable_height(0.01)      # Store only depth > 1cm
domain.set_flow_algorithm('DE0')

##########################################################################
#
# Using discontinuous elevation solver DE0
#
# First order timestepping
#
# Make sure you use centroid values when reporting on important output quantities
#
##########################################################################



## Setup initial condition and bathymetry


In [23]:
tide = 0.0
domain.set_quantity('stage', tide)
domain.set_quantity('friction', 0.0)


domain.set_quantity('elevation', 
                    filename='cairns.pts',
                    use_cache=cache,
                    verbose=verbose,
                    alpha=0.1)
time01 = time.time()
print 'That took %.2f seconds to fit data' %(time01-time00)

FitInterpolate: Building quad tree
Building smoothing matrix
Fit.fit: Initializing
Geospatial_data: Created from file: cairns.pts
Data will be loaded blockwise on demand
Got 1 variables: [u'elevation']
Default units is m.
ANUGA does not correct for differences in units.
Geospatial_data: Reading 1863558 points (in 2 block(s)) from file cairns.pts. 
Geospatial_data: Each block consists of 1000000 data points

Geospatial_data: Reading block 0 (points 0 to 1000000) out of 1
.
Geospatial_data: Reading block 1 (points 1000000 to 1863558) out of 1
 . 
Fit.fit: Warning: no data points in fit
Applying fitted data to domain
That took 539.86 seconds to fit data


## Setup boundary conditions

In [24]:
print 'Available boundary tags', domain.get_boundary_tags()

Bd = anuga.Dirichlet_boundary([tide, 0, 0]) # Mean water level
Bs = anuga.Transmissive_stage_zero_momentum_boundary(domain) # Neutral boundary

Available boundary tags ['ocean_east', 'top', 'onshore', 'bottom']


Huge 50m wave starting after 60 seconds and lasting 1 hour.

In [25]:
Bw = anuga.Transmissive_n_momentum_zero_t_momentum_set_stage_boundary(
                    domain=domain, 
                    function=lambda t: [(60<t<3660)*10, 0, 0])

domain.set_boundary({'ocean_east': Bw,
                     'bottom': Bs,
                     'onshore': Bd,
                     'top': Bs})

# Evolve system through time

In [ ]:
import time
t0 = time.time()

from numpy import allclose

# Save every two mins leading up to wave approaching land
for t in domain.evolve(yieldstep=2*60, finaltime=5000): 
    print domain.timestepping_statistics()
    print domain.boundary_statistics(tags='ocean_east')    

# Save every 30 secs as wave starts inundating ashore
for t in domain.evolve(yieldstep=60*0.5, finaltime=10000, 
                       skip_initial_step=True):
    print domain.timestepping_statistics()
    print domain.boundary_statistics(tags='ocean_east')
            
print 'That took %.2f seconds' %(time.time()-t0)